In [203]:
# Objects for defining the Hamiltonian
from SymPT import RDSymbol, RDBasis, BosonOp, Dagger, Block, Operator
# Objects for obtaining the Effective Hamiltonian
from SymPT import EffectiveFrame
# Built-in symbols
from SymPT import hbar, t
# Useful functions
from SymPT import display_dict, group_by_operators

import sympy as sp

# Setup

In [151]:
# ---------------- Defining the symbols ------------------
# Order 0
omega = RDSymbol('omega', positive=True, real=True)
omega_z = RDSymbol('omega_z', positive=True, real=True)
omega_d = RDSymbol('omega_d', positive=True, real=True)
m = RDSymbol('m', positive=True, real=True)
e = RDSymbol('e', positive=True, real=True)

# Order 1
bsl = RDSymbol('{b}_{SL}', order=1, positive=True, real=True)
E0 = RDSymbol('E0', order=1, positive=True, real=True)

# ----------------- Defining the basis -------------------
# Spin basis: Finite 2x2 Hilbert space
Spin = RDBasis('sigma', dim=2)
s0, sx, sy, sz = Spin.basis
# Position basis: Infinite bosonic Hilbert space
x = Operator('x')
px = Operator('p_x')
# Boson basis: Infinite bosonic Hilbert space
a = BosonOp('a')
ad = Dagger(a)
# Substituting the bosonic basis
subs_momentum_position = {
    x: sp.sqrt(hbar / (2 * m * omega)) * (a + ad),
    px: sp.I * sp.sqrt(hbar * m * omega / 2) * (ad - a)
}

# -------------- Defining the Hamiltonian ----------------
# Unperturbed Hamiltonian H0
H0_mp = px**2/(2 * m) + m * omega**2 * x**2 / 2 - hbar/2 * omega_z * sz
# Perturbation Hamiltonians
V_mp = -sp.Rational(1,2) * hbar * bsl * x * sx
Hd_mp = -e*E0 * sp.cos(omega_d * t) * x
display_dict({
    sp.Symbol('H_0'): H0_mp,
    sp.Symbol('V'): V_mp,
    sp.Symbol('H_d'): Hd_mp
})

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

# Time Independent Lab Frame and then rotate

In [153]:
# LabFrame Hamiltonian
H0_lab = H0_mp.subs(subs_momentum_position).expand()
V_lab = V_mp.subs(subs_momentum_position)
Hd_lab = Hd_mp.subs(subs_momentum_position)

display_dict({
    sp.Symbol('H_0'): H0_lab,
    sp.Symbol('V'): V_lab,
    sp.Symbol('H_d'): Hd_lab
})

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [189]:
bsl_tilde = sp.Symbol(r'\tilde{b}_{\mathrm{SL}}', real=True)
subs_bsl = {
    bsl: bsl_tilde * sp.sqrt(2 * m * omega / hbar)
}

E0_tilde = sp.Symbol(r'\tilde{E}_0', real=True)
subs_E0 = {
    E0: E0_tilde * sp.sqrt(2 * m * omega / hbar)
}

In [175]:
# Deffining Effective Hamiltonian Object
Eff_frame_lab = EffectiveFrame(H0_lab, V_lab, subspaces=[Spin])

The EffectiveFrame object has been initialized successfully.

Effective Frame

╭────────┬─────────┬─────────────╮
│  Name  │  Type   │  Dimension  │
├────────┼─────────┼─────────────┤
│ sigma  │ Finite  │     2x2     │
├────────┼─────────┼─────────────┤
│   a    │ Bosonic │      ∞      │
╰────────┴─────────┴─────────────╯

Effective Hamiltonian: 	Not computed yet. To do so, run `solve` method. 




In [176]:
# Calculate the effective model using the Schrieffer-Wolff transformation up to the second order
Eff_frame_lab.solve(max_order=2, full_diagonalization=True, mask=None)
# Obtaining the result in the operator form
H_eff_SWT_lab = Eff_frame_lab.get_H(return_form='operator')
group_by_operators(H_eff_SWT_lab)

The perturbative interaction will be added to the full Hamiltonian


Solving for each order:   0%|          | 0/2 [00:00<?, ?it/s]

Solving for each order: 100%|██████████| 2/2 [00:00<00:00,  7.32it/s]


The Hamiltonian has been solved successfully. Please use the get_H method to get the result in the desired form.


Projecting to operator form: 100%|██████████| 2/2 [00:00<00:00, 70.88it/s]


{sigma_3: hbar**2*omega_z*{b}_{SL}**2/(8*m*omega**3 - 8*m*omega*omega_z**2) - hbar*omega_z/2,
 sigma_3*Dagger(a)*a: hbar**2*omega_z*{b}_{SL}**2/(4*m*omega**3 - 4*m*omega*omega_z**2),
 1: -hbar**2*{b}_{SL}**2/(8*m*omega**2 - 8*m*omega_z**2) + hbar*omega/2,
 Dagger(a)*a: hbar*omega}

In [183]:
lx = sp.Symbol('l_x')
group_by_operators(Eff_frame_lab.corrections[2])[sz*ad*a].simplify().subs(subs_bsl)

hbar*omega_z*\tilde{b}_{\mathrm{SL}}**2/(2*(omega**2 - omega_z**2))

### Rotate Drive

In [199]:
H_Drive_lab = Eff_frame_lab.rotate(Hd_lab)
H_Drive_lab.subs(subs_E0).subs(subs_bsl).collect(2*m)

Projecting to operator form: 100%|██████████| 3/3 [00:00<00:00, 493.83it/s]


-e*omega*\tilde{E}_0*\tilde{b}_{\mathrm{SL}}*cos(omega_d*t)*sigma_1/(omega**2 - omega_z**2) - e*\tilde{E}_0*cos(omega_d*t)*Dagger(a) - e*\tilde{E}_0*cos(omega_d*t)*a

In [197]:
# Expectation Values for the bosonic operators
# <n=0|a|n=0> = 0
# <n=0|ad|n=0> = 0
# <n=0|ad*a|n=0> = 0
subs_expvals = {
    ad*a: 0,
    a : 0,
    ad : 0,
}
H_qubit_TI_lab = (H_eff_SWT_lab + H_Drive_lab).xreplace(subs_expvals)
H_qubit_TI_lab_dict = group_by_operators(H_qubit_TI_lab)
H_qubit_TI_lab_sz = H_qubit_TI_lab_dict[sz].subs(subs_bsl).subs(subs_E0).collect(2*m*omega)
H_qubit_TI_lab_sx = H_qubit_TI_lab_dict[sx].subs(subs_bsl).subs(subs_E0).collect(2*m)
H_qubit_TI_lab_sy = H_qubit_TI_lab_dict.get(sy, 0)

H_qubit_TI_lab = H_qubit_TI_lab_sz * sz + H_qubit_TI_lab_sx * sx + H_qubit_TI_lab_sy * sy
H_qubit_TI_lab

-e*omega*\tilde{E}_0*\tilde{b}_{\mathrm{SL}}*cos(omega_d*t)*sigma_1/(omega**2 - omega_z**2) + (hbar*omega_z*\tilde{b}_{\mathrm{SL}}**2/(4*omega**2 - 4*omega_z**2) - hbar*omega_z/2)*sigma_3

In [192]:
Rabi_Freq_lab = (H_qubit_TI_lab_sx / hbar).collect(2*m)
Rabi_Freq_lab.subs(subs_bsl).subs(subs_E0)

-e*omega*\tilde{E}_0*\tilde{b}_{\mathrm{SL}}*cos(omega_d*t)/(hbar*(omega**2 - omega_z**2))

# Time Independent Co-moving Frame and then rotate

In [200]:
# Co-moving Frame Hamiltonian
d = RDSymbol('d', positive=True, real=True)
H0_mp_co = H0_mp.subs({x: x + d})
V_mp_co  = V_mp.subs({x: x + d})
Hd_mp_co = Hd_mp.subs({x: x + d})
d_val = sp.solve((H0_mp_co + Hd_mp_co).expand().coeff(x), d)[0]
display(d_val)

H0_co = H0_mp.subs(subs_momentum_position).expand()
V_co = V_mp.subs(subs_momentum_position)
Hd_co =  Hd_mp_co.subs({x:0, d: d_val}).expand() + sp.diff(d_val, t) * px.subs(subs_momentum_position) + V_mp_co.subs({x:0, d: d_val}) + (m*omega**2/2 * d**2).subs({d: d_val})

display_dict({
    sp.Symbol('H_0'): H0_mp_co,
    sp.Symbol('V'): V_mp_co,
    sp.Symbol('H_d'): Hd_mp_co
})

# ----------------- Effective Hamiltonian ----------------
display_dict({
    sp.Symbol('H_0'): H0_co,
    sp.Symbol('V'): V_co,
    sp.Symbol('H_d'): Hd_co
})

E0*e*cos(omega_d*t)/(m*omega**2)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [201]:
# Deffining Effective Hamiltonian Object
Eff_frame_co = EffectiveFrame(H0_co, V_co, subspaces=[Spin])

The EffectiveFrame object has been initialized successfully.

Effective Frame

╭────────┬─────────┬─────────────╮
│  Name  │  Type   │  Dimension  │
├────────┼─────────┼─────────────┤
│ sigma  │ Finite  │     2x2     │
├────────┼─────────┼─────────────┤
│   a    │ Bosonic │      ∞      │
╰────────┴─────────┴─────────────╯

Effective Hamiltonian: 	Not computed yet. To do so, run `solve` method. 




In [ ]:
# Calculate the effective model using the Schrieffer-Wolff transformation up to the second order
Eff_frame_co.solve(max_order=2, full_diagonalization=True, mask=None)
# Obtaining the result in the operator form
H_eff_SWT_co = Eff_frame_co.get_H(return_form='operator')
H_eff_SWT_co

The perturbative interaction will be added to the full Hamiltonian


Solving for each order: 100%|██████████| 2/2 [00:00<00:00,  7.17it/s]


The Hamiltonian has been solved successfully. Please use the get_H method to get the result in the desired form.


Projecting to operator form: 100%|██████████| 2/2 [00:00<00:00, 68.20it/s]


hbar**2*omega_z*{b}_{SL}**2*sigma_3/(8*m*omega**3 - 8*m*omega*omega_z**2) + hbar**2*omega_z*{b}_{SL}**2*sigma_3*Dagger(a)*a/(4*m*omega**3 - 4*m*omega*omega_z**2) - hbar**2*{b}_{SL}**2/(8*m*omega**2 - 8*m*omega_z**2) + hbar*omega/2 + hbar*omega*Dagger(a)*a - hbar*omega_z*sigma_3/2

The perturbative interaction will be added to the full Hamiltonian



KeyboardInterrupt



In [ ]:
H_Drive_co = Eff_frame_co.rotate(Hd_co)
H_Drive_co

Projecting to operator form: 100%|██████████| 3/3 [00:00<00:00, 125.50it/s]


-E0**2*e**2*cos(omega_d*t)**2/(2*m*omega**2) - sqrt(2)*I*E0*e*sqrt(hbar)*omega_d*sin(omega_d*t)*Dagger(a)/(2*sqrt(m)*omega**(3/2)) + sqrt(2)*I*E0*e*sqrt(hbar)*omega_d*sin(omega_d*t)*a/(2*sqrt(m)*omega**(3/2)) - E0*e*hbar*omega_d*omega_z*{b}_{SL}*sin(omega_d*t)*sigma_2/(2*m*omega**4 - 2*m*omega**2*omega_z**2) - E0*e*hbar*{b}_{SL}*cos(omega_d*t)*sigma_1/(2*m*omega**2)

In [164]:
# Expectation Values for the bosonic operators
# <n=0|a|n=0> = 0
# <n=0|ad|n=0> = 0
# <n=0|ad*a|n=0> = 0
subs_expvals = {
    ad*a: 0,
    a : 0,
    ad : 0,
}
H_qubit_TI_co = (H_eff_SWT_co + H_Drive_co).subs(subs_expvals)
H_qubit_TI_co_dict = group_by_operators(H_qubit_TI_co)
H_qubit_TI_co_sz = H_qubit_TI_co_dict[sz].collect(2*m*omega).collect(sp.Rational(1,2)*hbar)
H_qubit_TI_co_sx = H_qubit_TI_co_dict[sx]
H_qubit_TI_co_sy = H_qubit_TI_co_dict.get(sy, 0).factor()

H_qubit_TI_co = H_qubit_TI_co_sz * sz + sp.factor_terms(H_qubit_TI_co_sx * sx + H_qubit_TI_co_sy * sy)
H_qubit_TI_co

-E0*e*hbar*{b}_{SL}*(omega_d*omega_z*sin(omega_d*t)*sigma_2/((omega - omega_z)*(omega + omega_z)) + cos(omega_d*t)*sigma_1)/(2*m*omega**2) + hbar*(hbar*omega_z*{b}_{SL}**2/(m*omega*(4*omega**2 - 4*omega_z**2)) - omega_z)*sigma_3/2

In [165]:
Rabi_Freq_co = sp.factor_terms(-H_qubit_TI_co_sx / hbar / sp.cos(omega_d*t)  + H_qubit_TI_co_sy/hbar / sp.sin(omega_d*t))
Rabi_Freq_co.factor()

E0*e*{b}_{SL}*(omega**2 - omega_d*omega_z - omega_z**2)/(2*m*omega**2*(omega - omega_z)*(omega + omega_z))

# Time Dependent

In [166]:
# Deffining Effective Hamiltonian Object
Eff_frame = EffectiveFrame(H0_lab, V_lab + Hd_lab, subspaces=[Spin])

The EffectiveFrame object has been initialized successfully.

Effective Frame

╭────────┬─────────┬─────────────╮
│  Name  │  Type   │  Dimension  │
├────────┼─────────┼─────────────┤
│ sigma  │ Finite  │     2x2     │
├────────┼─────────┼─────────────┤
│   a    │ Bosonic │      ∞      │
╰────────┴─────────┴─────────────╯

Effective Hamiltonian: 	Not computed yet. To do so, run `solve` method. 




In [167]:
# Defining the mask
mask = Block(fin=sx, inf=a, subspaces=[Spin]) + Block(fin=s0, inf=a, subspaces=[Spin]) + Block(fin=s0, inf=a**2, subspaces=[Spin])
# Calculate the effective model using the Schrieffer-Wolff transformation up to the second order
Eff_frame.solve(max_order=2, full_diagonalization=False, mask=mask)
# Obtaining the result in the operator form
H_eff_SWT = Eff_frame.get_H(return_form='matrix')

The perturbative interaction will be added to the full Hamiltonian


Solving for each order:   0%|          | 0/2 [00:00<?, ?it/s]

Solving for each order: 100%|██████████| 2/2 [00:00<00:00,  4.55it/s]


The Hamiltonian has been solved successfully. Please use the get_H method to get the result in the desired form.


Converting to matrix form: 100%|██████████| 2/2 [00:00<00:00, 1057.03it/s]


In [168]:
# Expectation Values for the bosonic operators
# <n=0|a|n=0> = 0
# <n=0|ad|n=0> = 0
# <n=0|ad*a|n=0> = 0
subs_expvals = {
    ad*a: 0,
    a : 0,
    ad : 0,
}
H_qubit_TD = H_eff_SWT.xreplace(subs_expvals)
H_qubit_TD_dict = group_by_operators(Spin.project(sp.expand_complex(sp.factor_terms(H_qubit_TD.expand())).trigsimp()))
H_qubit_TD_sz = (H_qubit_TD_dict[sz] - hbar * omega_z / 2).cancel().collect(2*m*omega) + hbar * omega_z / 2
H_qubit_TD_sx = sp.factor_terms(H_qubit_TD_dict[sx].factor()).collect(E0*bsl/(2*m))
H_qubit_TD_sy = H_qubit_TD_dict.get(sy, 0)

H_qubit_TD = H_qubit_TD_sz * sz + H_qubit_TD_sx * sx + H_qubit_TD_sy * sy
H_qubit_TD

-E0*e*hbar*{b}_{SL}*(2*omega**2 - omega_d**2 - omega_z**2)*cos(omega_d*t)*sigma_1/(4*m*(omega - omega_d)*(omega + omega_d)*(omega - omega_z)*(omega + omega_z)) + (hbar*omega_z/2 + (hbar**2*omega_z*{b}_{SL}**2 + 2*m*omega*(-4*hbar*omega**2*omega_z + 4*hbar*omega_z**3))/(2*m*omega*(4*omega**2 - 4*omega_z**2)))*sigma_3

In [169]:
(H_qubit_TD_sx / hbar / sp.cos(omega_d*t)).factor()

-E0*e*{b}_{SL}*(2*omega**2 - omega_d**2 - omega_z**2)/(4*m*(omega - omega_d)*(omega + omega_d)*(omega - omega_z)*(omega + omega_z))